In [1]:
pip install holidays 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install workalendar

Note: you may need to restart the kernel to use updated packages.


In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

import os,sys
from scipy import stats
import numpy as np
import datetime as dt
import holidays

from pandas.tseries.holiday import AbstractHolidayCalendar, GoodFriday, Holiday, Easter, Day
from pandas.tseries.offsets import CustomBusinessDay

import time

from datetime import timedelta

# 1.Banco de Dados

* ## 1.1 Carregando conjunto de dados

In [4]:
dados = pd.read_excel("../An-lise-das-Vendas/Vendas_LojaCosmeticos.xlsx")


* ## 1.2 Análise dos Dados

### Visualização das 5 primeiras observações

In [5]:
dados.head(5)

,Data,Empresa,Departamento,Vendas
0,2018-01-02,Loja1,Tintas para cabelo,44.70
1,2018-01-02,Loja1,Manicure e Pedicure,355.13
2,2018-01-02,Loja1,Manicure e Pedicure,81.41
3,2018-01-02,Loja1,Manicure e Pedicure,207.08
4,2018-01-02,Loja1,Manicure e Pedicure,38.89


### Dimensão da Base de Dados

In [6]:
dados.shape

(411943, 4)

### Tipo das variáveis contidas na Base de Dados

In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411943 entries, 0 to 411942
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Data          411943 non-null  datetime64[ns]
 1   Empresa       411943 non-null  object        
 2   Departamento  411943 non-null  object        
 3   Vendas        411943 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.6+ MB


### Primeiro e Último Período contido na base de Dados

In [8]:
PeriodoInicio = dados['Data'].min()
PeriodoFim = dados['Data'].max()

print(f"Período Início: {PeriodoInicio} e Período Fim: {PeriodoFim }" )

Período Início: 2018-01-02 00:00:00 e Período Fim: 2021-01-02 00:00:00


### Verificando se há dados faltante

In [9]:
date_list = pd.date_range(start=PeriodoInicio, end=PeriodoFim + timedelta(days = 1), freq = 'D', closed= 'left')   ## Atenção: A data considerada em end não é considerada.

date_list = pd.DataFrame([date_list]).T.rename(columns={0:'Data'})

dados = pd.merge(dados, date_list, on=["Data"], how="right")

dados

,Data,Empresa,Departamento,Vendas
0,2018-01-02,Loja1,Tintas para cabelo,44.70
1,2018-01-02,Loja1,Manicure e Pedicure,355.13
2,2018-01-02,Loja1,Manicure e Pedicure,81.41
3,2018-01-02,Loja1,Manicure e Pedicure,207.08
4,2018-01-02,Loja1,Manicure e Pedicure,38.89
...,...,...,...,...
411947,2021-01-02,Loja1,Manicure e Pedicure,372.92
411948,2021-01-02,Loja1,Manicure e Pedicure,1067.32
411949,2021-01-02,Loja1,Manicure e Pedicure,413.82
411950,2021-01-02,Loja1,Manicure e Pedicure,59.37


### Dados faltantes

In [10]:
faltantes = dados[dados['Vendas'].isnull()]
faltantes

,Data,Empresa,Departamento,Vendas
111206,2018-10-21,NaN,NaN,NaN
135513,2018-12-25,NaN,NaN,NaN
137823,2019-01-01,NaN,NaN,NaN
272860,2019-12-25,NaN,NaN,NaN
275145,2020-01-01,NaN,NaN,NaN
330905,2020-05-28,NaN,NaN,NaN
346771,2020-07-10,NaN,NaN,NaN
409395,2020-12-25,NaN,NaN,NaN
411634,2021-01-01,NaN,NaN,NaN


### Adição dados faltantes ao modelo

Nas Vendas com valor vazio foi adicionado 0.01, pois para fazer a modelagem de séries temporais é necessário ter todos os dias do período de estudo no conjunto de dados

In [11]:
## Substituir onde tem valor faltante por um número ou caracter

dados['Vendas'] .fillna(0.01, inplace=True)
dados['Empresa'] .fillna(method = 'ffill', inplace=True)
dados['Departamento'].fillna("Vazio", inplace=True)

In [12]:
dados[dados['Departamento'] == "Vazio"]

,Data,Empresa,Departamento,Vendas
111206,2018-10-21,Loja1,Vazio,0.01
135513,2018-12-25,Loja1,Vazio,0.01
137823,2019-01-01,Loja1,Vazio,0.01
272860,2019-12-25,Loja1,Vazio,0.01
275145,2020-01-01,Loja1,Vazio,0.01
330905,2020-05-28,Loja1,Vazio,0.01
346771,2020-07-10,Loja1,Vazio,0.01
409395,2020-12-25,Loja1,Vazio,0.01
411634,2021-01-01,Loja1,Vazio,0.01


### Dummy dados faltantes

In [13]:
dados['UnidadeNãoAbre'] = np.where(dados['Vendas'] == 0.01, 1, 0)

In [14]:
dados.loc[111204: 111208]

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre
111204,2018-10-20,Loja1,Manicure e Pedicure,1437.07,0
111205,2018-10-20,Loja1,Manicure e Pedicure,1621.32,0
111206,2018-10-21,Loja1,Vazio,0.01,1
111207,2018-10-22,Loja1,Manicure e Pedicure,1395.52,0
111208,2018-10-22,Loja1,Manicure e Pedicure,61.16,0


* ## 1.3 Inclusão de variáveis no conjunto de dados

* ### Datas que desejo a previsão

In [15]:
date_list = pd.date_range(start= PeriodoFim + timedelta(days = 1), end='2021-03-01', freq = 'D', closed= 'left') ## Atualizar sempre para as datas que desejar a previsão

datas_pred = pd.DataFrame([date_list]).T.rename(columns={0:'Data'})

dados = pd.merge(dados, datas_pred, on=["Data"], how="outer")

* ### Extraindo dia, mês e ano da variável data conjunto de dados

In [17]:
dados['dia'] = dados['Data'].dt.day
dados['mês'] = dados['Data'].dt.month
dados['ano'] = dados['Data'].dt.year

In [18]:
## Verificando se todas as variáveis estão com o tipo correto.

dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412009 entries, 0 to 412008
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Data            412009 non-null  datetime64[ns]
 1   Empresa         411952 non-null  object        
 2   Departamento    411952 non-null  object        
 3   Vendas          411952 non-null  float64       
 4   UnidadeNãoAbre  411952 non-null  float64       
 5   dia             412009 non-null  int64         
 6   mês             412009 non-null  int64         
 7   ano             412009 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 28.3+ MB


* ### Determinando o nome e dia da semana

<p> Pode utilizar o nome do dia da semana para estabelecer a numeração dos dias da semana.
    No conjunto de dados em estudo considerei:
    
    Domingo = 1
    Segunda = 2
    Terça = 3
    Quarta = 4
    Quinta = 5 
    Sexta = 6
    Sábado = 7


In [20]:
dados['Dia_da_Semana'] = dados['Data'].dt.day_name().replace(["Sunday", "Monday", "Tuesday", "Wednesday", 
                                                              "Thursday", "Friday", "Saturday"],
                                                            [1, 2, 3, 4, 5, 6, 7])


* ### Numeração Semana do mês

In [22]:
conditions = [
    (dados['dia'] /7 <= 1),
    ((dados['dia'] /7 > 1) & (dados['dia'] /7 <= 2)),
    ((dados['dia'] /7 > 2) & (dados['dia'] /7 <= 3)),
    ((dados['dia'] /7 > 3) & (dados['dia'] /7 <= 4)),
    (dados['dia'] /7 > 4)]
choices = [1, 2, 3, 4, 5]

dados['Semana_Mês'] = np.select(conditions, choices, default= 0)

* ### Adicionando variável Feriado no conjunto de dados

In [30]:
class Feriados_Brasil(AbstractHolidayCalendar):
    rules = [
        Holiday('Confraternização Universal', month=1, day=1),
        # Terça feira de carnaval (47 dias antes da Páscoa)
        Holiday('Segunda Feira de Carnaval', month=1, day=1, offset=[Easter(), Day(-48)]),
        Holiday('Terça Feira de Carnaval', month=1, day=1, offset=[Easter(), Day(-47)]),
        # Sexta feira Santa(GoodFriday)
        GoodFriday,       
        Holiday('Tiradentes', month = 4, day = 21),
        Holiday('Dia do Trabalho', month = 5, day = 1),
        # Corpus Christi (60 dias após a Páscoa, ou 62 dias após a Sexta Feira Santa)
        Holiday('Corpus Christi', month=1, day=1, offset=[Easter(), Day(60)]),        
        Holiday('Independência do Brasil', month = 9, day = 7),
        Holiday('Nossa Senhora Aparecida - Padroeira do Brasil', month = 10, day = 12),
        Holiday('Finados', month = 11, day = 2),
        Holiday('Proclamação da República', month = 11, day = 15),
        Holiday('Natal', month = 12, day = 25)]
    
br_feriados = CustomBusinessDay(calendar=Feriados_Brasil())    
inst = Feriados_Brasil()

feriados = inst.holidays(dt.datetime(2018, 1, 1), dt.datetime(2021, 12, 31))
print(f"Feriados contidos na base de dados:\n\n {feriados}")

Feriados contidos na base de dados:

 DatetimeIndex(['2018-01-01', '2018-02-12', '2018-02-13', '2018-03-30',
               '2018-04-21', '2018-05-01', '2018-05-31', '2018-09-07',
               '2018-10-12', '2018-11-02', '2018-11-15', '2018-12-25',
               '2019-01-01', '2019-03-04', '2019-03-05', '2019-04-19',
               '2019-04-21', '2019-05-01', '2019-06-20', '2019-09-07',
               '2019-10-12', '2019-11-02', '2019-11-15', '2019-12-25',
               '2020-01-01', '2020-02-24', '2020-02-25', '2020-04-10',
               '2020-04-21', '2020-05-01', '2020-06-11', '2020-09-07',
               '2020-10-12', '2020-11-02', '2020-11-15', '2020-12-25',
               '2021-01-01', '2021-02-15', '2021-02-16', '2021-04-02',
               '2021-04-21', '2021-05-01', '2021-06-03', '2021-09-07',
               '2021-10-12', '2021-11-02', '2021-11-15', '2021-12-25'],
              dtype='datetime64[ns]', freq=None)


In [33]:
feriados = pd.DataFrame(feriados)
 
## Renomeando a coluna com os feriados para Dia

feriados = feriados.rename(columns = {0:'Data'})


In [45]:
juncao2 = dados.merge(feriados, on='Data', how='left')

dados = juncao2.assign(feriados = lambda dataframe: dataframe['_merge'].map(lambda _merge: 1 if _merge == "both" else 0))


In [51]:
dados.loc[15651:15657]

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,_merge,feriados
15651,2018-02-11,Loja1,Manicure e Pedicure,124.40,0.0,11,2,2018,1,2,left_only,0
15652,2018-02-11,Loja1,Manicure e Pedicure,788.10,0.0,11,2,2018,1,2,left_only,0
15653,2018-02-11,Loja1,Manicure e Pedicure,184.96,0.0,11,2,2018,1,2,left_only,0
15654,2018-02-12,Loja1,Manicure e Pedicure,784.78,0.0,12,2,2018,2,2,both,1
15655,2018-02-12,Loja1,Manicure e Pedicure,75.23,0.0,12,2,2018,2,2,both,1
15656,2018-02-12,Loja1,Manicure e Pedicure,208.01,0.0,12,2,2018,2,2,both,1
15657,2018-02-12,Loja1,Manicure e Pedicure,93.97,0.0,12,2,2018,2,2,both,1


* ### Adição da Variável Domingo

In [54]:
dados['Domingo'] = np.where(dados["Dia_da_Semana"] == 1, 1, 0)

dados

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,_merge,feriados,Domingo
0,2018-01-02,Loja1,Tintas para cabelo,44.70,0.0,2,1,2018,3,1,left_only,0,0
1,2018-01-02,Loja1,Manicure e Pedicure,355.13,0.0,2,1,2018,3,1,left_only,0,0
2,2018-01-02,Loja1,Manicure e Pedicure,81.41,0.0,2,1,2018,3,1,left_only,0,0
3,2018-01-02,Loja1,Manicure e Pedicure,207.08,0.0,2,1,2018,3,1,left_only,0,0
4,2018-01-02,Loja1,Manicure e Pedicure,38.89,0.0,2,1,2018,3,1,left_only,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
412004,2021-02-24,NaN,NaN,NaN,NaN,24,2,2021,4,4,left_only,0,0
412005,2021-02-25,NaN,NaN,NaN,NaN,25,2,2021,5,4,left_only,0,0
412006,2021-02-26,NaN,NaN,NaN,NaN,26,2,2021,6,4,left_only,0,0
412007,2021-02-27,NaN,NaN,NaN,NaN,27,2,2021,7,4,left_only,0,0


* #### Adição da Variável Dias Não Úteis

mesclando a coluna feriado com domingo para obter uma coluna onde é atribuido 0 aos dias uteis e 1 aos domingos e feriados.

In [57]:
dados['DiasNaoUteis'] = dados['feriados'] | dados['Domingo']

In [62]:
dados.loc[411995:4120007]

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,_merge,feriados,Domingo,DiasNaoUteis
411995,2021-02-15,NaN,NaN,NaN,NaN,15,2,2021,2,3,both,1,0,1
411996,2021-02-16,NaN,NaN,NaN,NaN,16,2,2021,3,3,both,1,0,1
411997,2021-02-17,NaN,NaN,NaN,NaN,17,2,2021,4,3,left_only,0,0,0
411998,2021-02-18,NaN,NaN,NaN,NaN,18,2,2021,5,3,left_only,0,0,0
411999,2021-02-19,NaN,NaN,NaN,NaN,19,2,2021,6,3,left_only,0,0,0
412000,2021-02-20,NaN,NaN,NaN,NaN,20,2,2021,7,3,left_only,0,0,0
412001,2021-02-21,NaN,NaN,NaN,NaN,21,2,2021,1,3,left_only,0,1,1
412002,2021-02-22,NaN,NaN,NaN,NaN,22,2,2021,2,4,left_only,0,0,0
412003,2021-02-23,NaN,NaN,NaN,NaN,23,2,2021,3,4,left_only,0,0,0
412004,2021-02-24,NaN,NaN,NaN,NaN,24,2,2021,4,4,left_only,0,0,0


* #### Adição da Variável com numeração de dia útil

Aparentemente o código só retorna a numeração dos dias úteis corretamente se a planilha tiver departamento e seção, caso contrário é com chegar.

In [64]:
dados.sort_values('Data', inplace=True) 

last_date_month = 0 #Váriavel que salva o último dia útil para checar se começou um mês novo
working_days = 0 #Contador de qual dia útil do mês nós estamos
last_day = 0
nth_working_day = [] #Onde vamos salvar a lista de dias úteis para depois criar a série de dias útei que queremos

for index, row in dados.iterrows(): ## Faz a iteração em linhas de DataFrame como pares (índice, série).
    

    if row['DiasNaoUteis'] == 1:
        #Esse código vai ser executado nos fds e feriados.
        nth_working_day.append(0)
    else:
        #Esse código vai ser executado quando for dia útil
        
        #Checa se é um mês novo e reseta o contador se for.
        if row['mês'] != last_date_month:
            last_date_month = row['mês']
            working_days = 0
            
        if row['Data'] != last_day:
            
        #Soma 1 ao contador de dias uteis e adiciona a nossa lista
            working_days += 1
        nth_working_day.append(working_days)
        last_day = row['Data']

dados['Dia_Úteis'] = nth_working_day

In [68]:
dados.tail(28)

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,_merge,feriados,Domingo,DiasNaoUteis,Dia_Úteis
411981,2021-02-01,NaN,NaN,NaN,NaN,1,2,2021,2,1,left_only,0,0,0,1
411982,2021-02-02,NaN,NaN,NaN,NaN,2,2,2021,3,1,left_only,0,0,0,2
411983,2021-02-03,NaN,NaN,NaN,NaN,3,2,2021,4,1,left_only,0,0,0,3
411984,2021-02-04,NaN,NaN,NaN,NaN,4,2,2021,5,1,left_only,0,0,0,4
411985,2021-02-05,NaN,NaN,NaN,NaN,5,2,2021,6,1,left_only,0,0,0,5
411986,2021-02-06,NaN,NaN,NaN,NaN,6,2,2021,7,1,left_only,0,0,0,6
411987,2021-02-07,NaN,NaN,NaN,NaN,7,2,2021,1,1,left_only,0,1,1,0
411988,2021-02-08,NaN,NaN,NaN,NaN,8,2,2021,2,2,left_only,0,0,0,7
411989,2021-02-09,NaN,NaN,NaN,NaN,9,2,2021,3,2,left_only,0,0,0,8
411990,2021-02-10,NaN,NaN,NaN,NaN,10,2,2021,4,2,left_only,0,0,0,9


* #### Adicionando variável do Pagamento no Quinto Dia Util

Uma das datas de pagamento é o quinto dia útil do mês, no entanto quando o quinto dia útil cai no sábado o pagamento é realizado no dia anterior.

Foi Atribuido 1 ao dia do pagamento referente do quinto dia útil

In [70]:
conditions = [
    (dados['Dia_Úteis'] == 5) & (dados['Dia_da_Semana'] != 7.0),
    (dados['Dia_Úteis'] == 4) & (dados['Dia_da_Semana'] == 6.0)]
choices = [1, 1]
dados['pagamento'] = np.select(conditions, choices, default= 0)

In [72]:
dados.tail(28)

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,_merge,feriados,Domingo,DiasNaoUteis,Dia_Úteis,pagamento
411981,2021-02-01,NaN,NaN,NaN,NaN,1,2,2021,2,1,left_only,0,0,0,1,0
411982,2021-02-02,NaN,NaN,NaN,NaN,2,2,2021,3,1,left_only,0,0,0,2,0
411983,2021-02-03,NaN,NaN,NaN,NaN,3,2,2021,4,1,left_only,0,0,0,3,0
411984,2021-02-04,NaN,NaN,NaN,NaN,4,2,2021,5,1,left_only,0,0,0,4,0
411985,2021-02-05,NaN,NaN,NaN,NaN,5,2,2021,6,1,left_only,0,0,0,5,1
411986,2021-02-06,NaN,NaN,NaN,NaN,6,2,2021,7,1,left_only,0,0,0,6,0
411987,2021-02-07,NaN,NaN,NaN,NaN,7,2,2021,1,1,left_only,0,1,1,0,0
411988,2021-02-08,NaN,NaN,NaN,NaN,8,2,2021,2,2,left_only,0,0,0,7,0
411989,2021-02-09,NaN,NaN,NaN,NaN,9,2,2021,3,2,left_only,0,0,0,8,0
411990,2021-02-10,NaN,NaN,NaN,NaN,10,2,2021,4,2,left_only,0,0,0,9,0


* ###  Adicionando variável do VALE

In [76]:
conditions = [
    (dados['dia'] == 19) & (dados['Dia_da_Semana'] == 6),   
    (dados['dia'] == 18) & (dados['Dia_da_Semana'] == 6),
    (dados['dia'] == 20) & (dados['Dia_da_Semana'] > 1) & (dados['Dia_da_Semana'] <= 6)]
choices = [1, 1, 1]
dados['Vale'] = np.select(conditions, choices, default= 0)

In [77]:
dados.tail(28)

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,_merge,feriados,Domingo,DiasNaoUteis,Dia_Úteis,pagamento,Vale
411981,2021-02-01,NaN,NaN,NaN,NaN,1,2,2021,2,1,left_only,0,0,0,1,0,0
411982,2021-02-02,NaN,NaN,NaN,NaN,2,2,2021,3,1,left_only,0,0,0,2,0,0
411983,2021-02-03,NaN,NaN,NaN,NaN,3,2,2021,4,1,left_only,0,0,0,3,0,0
411984,2021-02-04,NaN,NaN,NaN,NaN,4,2,2021,5,1,left_only,0,0,0,4,0,0
411985,2021-02-05,NaN,NaN,NaN,NaN,5,2,2021,6,1,left_only,0,0,0,5,1,0
411986,2021-02-06,NaN,NaN,NaN,NaN,6,2,2021,7,1,left_only,0,0,0,6,0,0
411987,2021-02-07,NaN,NaN,NaN,NaN,7,2,2021,1,1,left_only,0,1,1,0,0,0
411988,2021-02-08,NaN,NaN,NaN,NaN,8,2,2021,2,2,left_only,0,0,0,7,0,0
411989,2021-02-09,NaN,NaN,NaN,NaN,9,2,2021,3,2,left_only,0,0,0,8,0,0
411990,2021-02-10,NaN,NaN,NaN,NaN,10,2,2021,4,2,left_only,0,0,0,9,0,0


* ### Numeração da semana do mês

##### 1º forma - Não foi utilizada na modelagem
    
Numeração da semana do mês levando em consideração que o mês tem início no domingo e fim no sábado, portanto caso a semana 
comece na quarta a primeira semana terá apenas 4 dias. Nesse caso é possível ter até a 6° semana, caso a semana comece na sexta e termine em um domingo, teremos: primeira semana com 2 dias e a ultima com apenas 1 dia, um exemplo.

##### 2º forma ( UTILIZADA NAS MODELAGENS)

Contagem dos dias da semana sempre de 7 em 7, independente do dia da semana que começa o mês.

In [78]:
conditions = [
    (dados['dia'] /7 <= 1),
    ((dados['dia'] /7 > 1) & (dados['dia'] /7 <= 2)),
    ((dados['dia'] /7 > 2) & (dados['dia'] /7 <= 3)),
    ((dados['dia'] /7 > 3) & (dados['dia'] /7 <= 4)),
    (dados['dia'] /7 > 4)]
choices = [1, 2, 3, 4, 5]

dados['Semana_Mês'] = np.select(conditions, choices, default= 0)

In [79]:
dados.tail(28)

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,_merge,feriados,Domingo,DiasNaoUteis,Dia_Úteis,pagamento,Vale
411981,2021-02-01,NaN,NaN,NaN,NaN,1,2,2021,2,1,left_only,0,0,0,1,0,0
411982,2021-02-02,NaN,NaN,NaN,NaN,2,2,2021,3,1,left_only,0,0,0,2,0,0
411983,2021-02-03,NaN,NaN,NaN,NaN,3,2,2021,4,1,left_only,0,0,0,3,0,0
411984,2021-02-04,NaN,NaN,NaN,NaN,4,2,2021,5,1,left_only,0,0,0,4,0,0
411985,2021-02-05,NaN,NaN,NaN,NaN,5,2,2021,6,1,left_only,0,0,0,5,1,0
411986,2021-02-06,NaN,NaN,NaN,NaN,6,2,2021,7,1,left_only,0,0,0,6,0,0
411987,2021-02-07,NaN,NaN,NaN,NaN,7,2,2021,1,1,left_only,0,1,1,0,0,0
411988,2021-02-08,NaN,NaN,NaN,NaN,8,2,2021,2,2,left_only,0,0,0,7,0,0
411989,2021-02-09,NaN,NaN,NaN,NaN,9,2,2021,3,2,left_only,0,0,0,8,0,0
411990,2021-02-10,NaN,NaN,NaN,NaN,10,2,2021,4,2,left_only,0,0,0,9,0,0


* #### Variável com a dummy dos Meses ( Não foi utilizada nas modelagens)

* ### Dummy dos Dias da Semana

#### 1º forma de fazer, usando pandas

In [82]:
## Variáveis dummies dias da semana (outra forma de fazer)

dados =   dados.assign(domingo = lambda dataframe: dataframe['Dia_da_Semana'].map(lambda Dia_da_Semana: 1 if Dia_da_Semana == 1 else 0))
dados =   dados.assign(segunda = lambda dataframe: dataframe['Dia_da_Semana'].map(lambda Dia_da_Semana: 1 if Dia_da_Semana == 2 else 0))
dados =   dados.assign(terça = lambda dataframe: dataframe['Dia_da_Semana'].map(lambda Dia_da_Semana: 1 if Dia_da_Semana == 3 else 0))
dados =   dados.assign(quarta = lambda dataframe: dataframe['Dia_da_Semana'].map(lambda Dia_da_Semana: 1 if Dia_da_Semana == 4 else 0))
dados =   dados.assign(quinta = lambda dataframe: dataframe['Dia_da_Semana'].map(lambda Dia_da_Semana: 1 if Dia_da_Semana == 5 else 0))
dados =   dados.assign(sexta = lambda dataframe: dataframe['Dia_da_Semana'].map(lambda Dia_da_Semana: 1 if Dia_da_Semana == 6 else 0))
dados =   dados.assign(sabado = lambda dataframe: dataframe['Dia_da_Semana'].map(lambda Dia_da_Semana: 1 if Dia_da_Semana == 7 else 0))


In [83]:
dados.tail(28)

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,...,Dia_Úteis,pagamento,Vale,domingo,segunda,terça,quarta,quinta,sexta,sabado
411981,2021-02-01,NaN,NaN,NaN,NaN,1,2,2021,2,1,...,1,0,0,0,1,0,0,0,0,0
411982,2021-02-02,NaN,NaN,NaN,NaN,2,2,2021,3,1,...,2,0,0,0,0,1,0,0,0,0
411983,2021-02-03,NaN,NaN,NaN,NaN,3,2,2021,4,1,...,3,0,0,0,0,0,1,0,0,0
411984,2021-02-04,NaN,NaN,NaN,NaN,4,2,2021,5,1,...,4,0,0,0,0,0,0,1,0,0
411985,2021-02-05,NaN,NaN,NaN,NaN,5,2,2021,6,1,...,5,1,0,0,0,0,0,0,1,0
411986,2021-02-06,NaN,NaN,NaN,NaN,6,2,2021,7,1,...,6,0,0,0,0,0,0,0,0,1
411987,2021-02-07,NaN,NaN,NaN,NaN,7,2,2021,1,1,...,0,0,0,1,0,0,0,0,0,0
411988,2021-02-08,NaN,NaN,NaN,NaN,8,2,2021,2,2,...,7,0,0,0,1,0,0,0,0,0
411989,2021-02-09,NaN,NaN,NaN,NaN,9,2,2021,3,2,...,8,0,0,0,0,1,0,0,0,0
411990,2021-02-10,NaN,NaN,NaN,NaN,10,2,2021,4,2,...,9,0,0,0,0,0,1,0,0,0


#### 2° forma de fazer

* ### Dummy Semana do mês

#### 1° forma de fazer

In [86]:
## Variáveis Semana do mês
dados = dados.assign(PrimeiraSemMes = lambda dataframe: dataframe['Semana_Mês'].map(lambda Semana_Mês: 1 if Semana_Mês == 1 else 0))
dados = dados.assign(SegundaSemMes = lambda dataframe: dataframe['Semana_Mês'].map(lambda Semana_Mês: 1 if Semana_Mês == 2 else 0))
dados = dados.assign(TerceiraSemMes = lambda dataframe: dataframe['Semana_Mês'].map(lambda Semana_Mês: 1 if Semana_Mês == 3 else 0))
dados = dados.assign(QuartaSemMes = lambda dataframe: dataframe['Semana_Mês'].map(lambda Semana_Mês: 1 if Semana_Mês == 4 else 0))
dados = dados.assign(QuintaSemMes = lambda dataframe: dataframe['Semana_Mês'].map(lambda Semana_Mês: 1 if Semana_Mês == 5 else 0))

In [87]:
dados.tail(28)

,Data,Empresa,Departamento,Vendas,UnidadeNãoAbre,dia,mês,ano,Dia_da_Semana,Semana_Mês,...,Primeira_SemMes,Segunda_SemMes,Terceira_SemMes,Quarta_SemMes,Quinta_SemMes,PrimeiraSemMes,SegundaSemMes,TerceiraSemMes,QuartaSemMes,QuintaSemMes
411981,2021-02-01,NaN,NaN,NaN,NaN,1,2,2021,2,1,...,0,1,0,0,0,1,0,0,0,0
411982,2021-02-02,NaN,NaN,NaN,NaN,2,2,2021,3,1,...,0,0,1,0,0,1,0,0,0,0
411983,2021-02-03,NaN,NaN,NaN,NaN,3,2,2021,4,1,...,0,0,0,1,0,1,0,0,0,0
411984,2021-02-04,NaN,NaN,NaN,NaN,4,2,2021,5,1,...,0,0,0,0,1,1,0,0,0,0
411985,2021-02-05,NaN,NaN,NaN,NaN,5,2,2021,6,1,...,0,0,0,0,0,1,0,0,0,0
411986,2021-02-06,NaN,NaN,NaN,NaN,6,2,2021,7,1,...,0,0,0,0,0,1,0,0,0,0
411987,2021-02-07,NaN,NaN,NaN,NaN,7,2,2021,1,1,...,1,0,0,0,0,1,0,0,0,0
411988,2021-02-08,NaN,NaN,NaN,NaN,8,2,2021,2,2,...,0,1,0,0,0,0,1,0,0,0
411989,2021-02-09,NaN,NaN,NaN,NaN,9,2,2021,3,2,...,0,0,1,0,0,0,1,0,0,0
411990,2021-02-10,NaN,NaN,NaN,NaN,10,2,2021,4,2,...,0,0,0,1,0,0,1,0,0,0


* ### Retirando as variáveis que não serão utilizadas na modelagem

In [90]:
dados = dados.drop(['_merge', 'Domingo', 'DiasNaoUteis', 'Dia_Úteis',  'Dia_da_Semana'], axis=1)

#### Criando Planilha no Excel com as variáveis Dummys 

In [91]:
dados.to_excel("Variaveis_ModeloPrediçao.xlsx")